Imported Libaries

In [1]:
import pandas as pd
import chardet
import os
from scipy.stats import zscore
pd.options.mode.chained_assignment = None
from sqlalchemy import create_engine, text, inspect
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from IPython.display import display as original_display
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns
import inspect
import re
import string
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

Standard Functions

In [2]:
#Function to clean labels in any plot functions
def clean_label(label):
    return label.replace('_', ' ').title()
 
#Function for getting the name of a Dataframe
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name
 
#Function to validate the data in a Dataframe
def validate_data(df, show_counts=True):
    df_name = get_var_name(df)
    print(f'#########################################################################################################################################################################################\nDataFrame: {df_name}')
    #snapshot the dataset
    display(df)
    #check for unique values
    unique_counts = pd.DataFrame(df.nunique())
    unique_counts = unique_counts.reset_index().rename(columns={0:'No. of Unique Values', 'index':'Field Name'})
    print("Unique values per field:")
    pd.set_option('display.max_rows', None)
    display(unique_counts)
    pd.reset_option('display.max_rows')
    #checking for duplicates
    duplicate_count = df.duplicated().sum()
    print("\nNumber of duplicate rows:")
    print(duplicate_count,'\n')
    info = df.info(show_counts=show_counts)
    display(info)
    #summary stats
    print("\nSummary statistics:")
    display(df.describe())
    print('End of data validation\n#########################################################################################################################################################################################\n')
 
#Function to provide list for data sources as a dataframe when conducting analysis
def header_list(df):
    df_list_ = df.copy()
    df_list = df_list_.columns.tolist()
    df_list = pd.DataFrame(df_list)
    new_header = df_list.iloc[0]  # Get the first row for the header
    df_list = df_list[1:]  # Take the data less the header row
    df_list.columns = new_header  # Set the header row as the df header
    df_list.reset_index(drop=True, inplace=True)  # Reset index
   
    return df_list
 
def query_data(schema, data):
    # Define the SQL query
    query = f'SELECT * FROM [{schema}].[{data}]'
 
    # Load data into DataFrame
    df = pd.read_sql(query, engine)
 
    print(f'Successfully imported {data}')
    # Display the DataFrame
    return df

def display(df):
    # Attempt to get the name of the DataFrame from the caller's local variables
    frame = inspect.currentframe().f_back
    # Attempt to find the variable name corresponding to the DataFrame
    name = "Unnamed DataFrame"
    for var_name, var_value in frame.f_locals.items():
        if var_value is df:
            name = var_name
            break
 
    # If the name is not in the list to be excluded, print it
    if name not in {'df', 'Unnamed DataFrame', 'unique_counts'}:
        print(f"DataFrame: {name}")
    # Always display the DataFrame regardless of the name
    original_display(df)

def unique_values(df, display_df=True):
    # Extract unique values for each field and store them in a dictionary
    unique_values = {col: df[col].unique() for col in df.columns}
    # Find the maximum number of unique values
    max_length = max(len(values) for values in unique_values.values())
    # Create a dictionary for the new DataFrame with padded None values
    unique_df_data = {}
    for col, values in unique_values.items():
        unique_df_data[col] = list(values) + [None] * (max_length - len(values))
    # Create the new DataFrame
    unique_df = pd.DataFrame(unique_df_data)
    if display_df == True:
        # Set display options to show all rows and display the DataFrame
        pd.set_option('display.max_rows', None)
        display(unique_df.head(100))
        # Reset display options back to default
        pd.reset_option('display.max_rows')

def export_to_csv(df):
    df_name = get_var_name(df)
    # Specify the directory and filename
    directory = r"C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry"
    file_path = f'{directory}\\{df_name}.csv'
    # Export the DataFrame to the specified directory
    df.to_csv(file_path, index=False)
    print(f'Successfully exported {df_name} to CSV')

In [3]:
def apply_daynight_features(df):
    dictionary = {
        '00-03':'6pm-6am',
        '03-06':'6pm-6am',
        '06-09':'6am-6pm',
        '09-12':'6am-6pm',
        '12-15':'6am-6pm',
        '15-18':'6am-6pm',
        '18-21':'6pm-6am',
        '21-24':'6pm-6am'
    }

    df['day_night'] = df['time_indicator'].map(dictionary)

    return df

def detect_anomalies(df, category,std=3,key=False,day_night=False,agg=False):
    if agg:
        agg = f'{agg}'
    else:
        agg='sum'
    categories = [
        'count_date','total_footfall',f'{category}_{agg}',
        'zscore','is_anomaly?'
    ]
    if key:
        categories = categories + [key]
    if day_night:
        categories = categories + ['day_night']

    anomalies = df.copy()
    anomalies['zscore'] = zscore(anomalies[f'{category}_{agg}'])
    anomalies['is_anomaly?'] = (anomalies['zscore'] < -std) | (anomalies['zscore'] > std)

    anomalies = anomalies[categories]
    anomalies['year'] = anomalies['count_date'].dt.year
    anomalies['moving_average'] = anomalies[f'{category}_{agg}'].rolling(window=7).mean()
    anomalies['corrected_value'] = np.where(
        anomalies['is_anomaly?'],
        anomalies['moving_average'],
        anomalies[f'{category}_{agg}']
    )
    anomalies['corrected_ma_monthly'] = anomalies['corrected_value'].rolling(window=30).mean()
    anomalies['corrected_ma_weekly'] = anomalies['corrected_value'].rolling(window=7).mean()

    return anomalies

def agg_footfall_data(df,category=False,day_night=False,agg=False,include_residents=False,workers_only=False,visitors_only=False):  
    merge_list = ['count_date']
    new_categories = [
        'count_date','corrected_ma_monthly',
        'corrected_ma_weekly','corrected_value'
    ]
    if day_night:
        merge_list = merge_list + ['day_night']
        new_categories = new_categories + ['day_night']
    if category:
        merge_list = merge_list + [category]
        new_categories = new_categories + [category]
    if agg:
        agg = f'{agg}'
    else:
        agg='sum'
    agg_data = df.groupby(merge_list).agg(
        total_footfall = ('total_footfall','sum'),
        residents_sum = ('resident','sum'),
        workers_sum = ('worker','sum'),
        visitors_sum = ('visitor','sum'),
        average_footfall = ('total_footfall','mean'),
        residents_mean = ('resident','mean'),
        workers_mean = ('worker','mean'),
        visitors_mean = ('visitor','mean'),
        loyalty = ('loyalty_percentage','mean'),
        dwell_time = ('dwell_time','mean')
    )

    agg_data = agg_data.reset_index()
    agg_data = agg_data.sort_values(
        ['count_date'],
        ascending=False
    )

    resident_z = detect_anomalies(agg_data,'residents',2.6,key=category,day_night=day_night,agg=agg)
    worker_z = detect_anomalies(agg_data,'workers',2.6,key=category,day_night=day_night,agg=agg)
    visitor_z = detect_anomalies(agg_data,'visitors',2.6,key=category,day_night=day_night,agg=agg)

    resident_merge = resident_z[new_categories]
    worker_merge = worker_z[new_categories]
    new_categories = new_categories + ['year']
    visitor_merge = visitor_z[new_categories]
    
    merge = pd.merge(
        resident_merge, worker_merge,
        how='left', on=merge_list,
        suffixes=['_res','_work']
    ).merge(
        visitor_merge,
        how='left', on=merge_list
    )

    if include_residents:
        merge['corrected_value_total'] = merge['corrected_value_res'] + merge['corrected_value_work'] + merge['corrected_value']
    elif workers_only:
        merge['corrected_value_total'] = merge['corrected_value_work']
    elif visitors_only:
        merge['corrected_value_total'] = merge['corrected_value']
    else:
        merge['corrected_value_total'] = merge['corrected_value_work'] + merge['corrected_value']
    
    merge['corrected_ma_monthly_total'] = merge['corrected_value_total'].rolling(window=30).mean()
    merge['corrected_ma_weekly_total'] = merge['corrected_value_total'].rolling(window=7).mean()

    return merge

def apply_features(df,date='count_date'):
    df['day_name'] = df[f'{date}'].dt.dayofweek

    dictionary = {
        '0':'Monday',
        '1':'Tuesday',
        '2':'Wednesday',
        '3':'Thursday',
        '4':'Friday',
        '5':'Saturday',
        '6':'Sunday'
    }
    weekend_dictionary = {
        '0':'Weekday',
        '1':'Weekday',
        '2':'Weekday',
        '3':'Weekday',
        '4':'Weekday',
        '5':'Weekend',
        '6':'Weekend'
    }
    df['day_name'] = df['day_name'].astype(str)
    df['week_name'] = df['day_name'].map(weekend_dictionary)
    df['day_name'] = df['day_name'].map(dictionary)


    return df

def transform_to_daynight(df,category):
    transform = df.copy()
    transform = transform.pivot_table(
        index = ['count_date','year','day_name','week_name'] + [category],
        columns='day_night',
        values='corrected_value_total'
    ).reset_index()

    return transform

Database and CWD setup and connection

In [4]:
# Database credentials
db_host = 'LBHHLWSQL0001.lbhf.gov.uk'
db_port = '1433'
db_name = 'IA_ODS'

# Create the connection string for SQL Server using pyodbc with Windows Authentication
connection_string = f'mssql+pyodbc://@{db_host}:{db_port}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes'

# Create the database engine
engine = create_engine(connection_string)

# Define the current working directory
cwd = r'C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Footfall and Spend Analysis\Footfall Data\LSOA Based'
os.chdir(cwd)
files = os.listdir(os.getcwd())
print("Files in %r: %s" % (cwd, files))

Files in 'C:\\Users\\jf79\\OneDrive - Office Shared Service\\Documents\\H&F Analysis\\Footfall and Spend Analysis\\Footfall Data\\LSOA Based': ['lsoa_hourly_counts_2022_H1.csv', 'lsoa_hourly_counts_2022_H2.csv', 'lsoa_hourly_counts_2024_H2.csv']


In [12]:
footfall_2024_Hex = pd.read_csv('C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data/Hex Based/Footfall Counts/hex_3hourly_counts_2024.csv')
relevant_hexes = pd.read_csv('C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data/Hex Based/Relevant Hexes/Relevant Hexes.csv')

In [9]:
display(footfall_2024_Hex)

DataFrame: footfall_2024_Hex


,hex_id,count_date,day,time_indicator,resident,visitor,worker,loyalty_percentage,dwell_time
0,10631211,2024-02-01,Thu,00-03,NaN,NaN,NaN,NaN,119.80
1,10631211,2024-02-01,Thu,03-06,NaN,NaN,NaN,NaN,72.83
2,10631211,2024-02-01,Thu,06-09,NaN,NaN,19.0,NaN,92.26
3,10761208,2024-05-16,Thu,15-18,NaN,NaN,NaN,NaN,19.87
4,10761208,2024-05-16,Thu,18-21,NaN,NaN,NaN,NaN,36.61
...,...,...,...,...,...,...,...,...,...
34958010,12541266,2024-10-19,Sat,21-24,NaN,NaN,NaN,NaN,NaN
34958011,12541266,2024-10-20,Sun,09-12,NaN,NaN,NaN,NaN,NaN
34958012,12541266,2024-10-20,Sun,12-15,NaN,NaN,NaN,NaN,NaN
34958013,12541266,2024-10-20,Sun,15-18,NaN,NaN,NaN,NaN,NaN


In [15]:
# footfall_2024_Hex['count_date'] = pd.to_datetime(footfall_2024_Hex['count_date'])
# footfall_2024_Hex = footfall_2024_Hex[
#     (footfall_2024_Hex['count_date'] > pd.to_datetime('2024-05-25')) &
#     (footfall_2024_Hex['count_date'] < pd.to_datetime('2024-06-29'))
# ]


,hex_id,count_date,day,time_indicator,resident,visitor,worker,loyalty_percentage,dwell_time
78,10761208,2024-05-26,Sun,00-03,NaN,NaN,NaN,NaN,39.94
79,10761208,2024-05-26,Sun,03-06,NaN,NaN,NaN,NaN,65.00
80,10761208,2024-05-26,Sun,06-09,NaN,NaN,NaN,NaN,33.08
81,10761208,2024-05-26,Sun,09-12,NaN,NaN,NaN,NaN,17.83
82,10761208,2024-05-26,Sun,12-15,NaN,NaN,NaN,NaN,20.45
...,...,...,...,...,...,...,...,...,...
29007442,12541266,2024-06-28,Fri,03-06,NaN,NaN,NaN,NaN,15.00
29007443,12541266,2024-06-28,Fri,09-12,NaN,NaN,NaN,NaN,25.00
29007444,12541266,2024-06-28,Fri,12-15,NaN,NaN,NaN,NaN,NaN
29007445,12541266,2024-06-28,Fri,18-21,NaN,NaN,NaN,NaN,54.71


In [16]:
footfall_data_2024 = footfall_2024_Hex.copy()
relevant_hexes_data = relevant_hexes.copy()

footfall_data_2024 = pd.merge(
    relevant_hexes_data,
    footfall_2024_Hex,
    left_on='Hex_ID',
    right_on='hex_id',
    how='left'
)
footfall_data_2024

,OID_,Col_ID,Row_ID,Hex_ID,Centroid_X,Centroid_Y,area,Shape_Length,Shape_Area,hex_id,count_date,day,time_indicator,resident,visitor,worker,loyalty_percentage,dwell_time
0,1,1120,1234,11201234,521148.5154,179650.7975,106088.115,521.032154,5283.915821,11201234,2024-05-26,Sun,00-03,716.0,98.0,46.0,NaN,152.98
1,1,1120,1234,11201234,521148.5154,179650.7975,106088.115,521.032154,5283.915821,11201234,2024-05-26,Sun,03-06,885.0,98.0,52.0,5.78,118.69
2,1,1120,1234,11201234,521148.5154,179650.7975,106088.115,521.032154,5283.915821,11201234,2024-05-26,Sun,06-09,755.0,94.0,46.0,NaN,142.51
3,1,1120,1234,11201234,521148.5154,179650.7975,106088.115,521.032154,5283.915821,11201234,2024-05-26,Sun,09-12,858.0,177.0,85.0,2.52,104.63
4,1,1120,1234,11201234,521148.5154,179650.7975,106088.115,521.032154,5283.915821,11201234,2024-05-26,Sun,12-15,743.0,156.0,56.0,2.84,100.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55749,205,1138,1218,11381218,526604.4754,176850.7975,106088.115,934.139762,57391.090939,11381218,2024-06-28,Fri,09-12,67.0,753.0,140.0,13.14,91.93
55750,205,1138,1218,11381218,526604.4754,176850.7975,106088.115,934.139762,57391.090939,11381218,2024-06-28,Fri,12-15,63.0,656.0,116.0,17.73,97.02
55751,205,1138,1218,11381218,526604.4754,176850.7975,106088.115,934.139762,57391.090939,11381218,2024-06-28,Fri,15-18,48.0,618.0,74.0,7.75,80.90
55752,205,1138,1218,11381218,526604.4754,176850.7975,106088.115,934.139762,57391.090939,11381218,2024-06-28,Fri,18-21,38.0,589.0,23.0,17.06,85.71


In [29]:
def quarterly_changes(start, end, q):
    footfall_data = footfall_data_2024.copy()
    print('Data copied')

    footfall_data = footfall_data.drop(
        columns = [
            'OID_','Col_ID','Row_ID','Hex_ID',
            'Centroid_X','Centroid_Y','area',
            'Shape_Length','Shape_Area'
        ]
    )
    footfall_data['count_date'] = pd.to_datetime(footfall_data['count_date'])

    footfall_data = footfall_data[
        (footfall_data['count_date'] < pd.to_datetime(end)) &
        (footfall_data['count_date'] > pd.to_datetime(start))]
    footfall_data = apply_daynight_features(footfall_data)
    columns_to_fill = [
        'resident','worker','visitor',
        'loyalty_percentage','dwell_time'
    ]
    footfall_data.loc[:, columns_to_fill] = footfall_data[columns_to_fill].applymap(lambda x: np.nan if x < 0 else x)
    footfall_data[columns_to_fill] = footfall_data[columns_to_fill].fillna(0)
    footfall_data['total_footfall'] = footfall_data['resident'] + footfall_data['worker'] + footfall_data['visitor']
    footfall_data = footfall_data.sort_values(by=['count_date','time_indicator','hex_id'])
    if q == 2:
        footfall_2024Q2 = footfall_data.copy()
        footfall_2024Q2 = agg_footfall_data(footfall_2024Q2,category='hex_id',day_night=True, agg='sum')
        footfall_2024Q2 = apply_features(footfall_2024Q2)
        footfall_2024Q2 = transform_to_daynight(footfall_2024Q2,'hex_id')
        final = footfall_2024Q2.copy()
        final = final.groupby(['year','week_name','hex_id']).agg(
            Daytime_sum = ('6am-6pm','mean'),
            Nighttime_sum = ('6pm-6am','mean')
        ).reset_index()
        weekdayQ2 = final[final['week_name'] == 'Weekday']
        weekendQ2 = final[final['week_name'] == 'Weekend']
        final = footfall_2024Q2.copy()
        typicalQ2 = final.groupby(['year','hex_id']).agg(
            Daytime_sum = ('6am-6pm','mean'),
            Nighttime_sum = ('6pm-6am','mean')
        ).reset_index()

        export_to_csv(typicalQ2)
    else:
        footfall_2024Q3 = footfall_data.copy()
        footfall_2024Q3 = agg_footfall_data(footfall_2024Q3,category='hex_id',day_night=True, agg='sum')
        footfall_2024Q3 = apply_features(footfall_2024Q3)
        footfall_2024Q3 = transform_to_daynight(footfall_2024Q3,'hex_id')
        final = footfall_2024Q3.copy()
        final = final.groupby(['year','week_name','hex_id']).agg(
            Daytime_sum = ('6am-6pm','mean'),
            Nighttime_sum = ('6pm-6am','mean')
        ).reset_index()
        weekdayQ3 = final[final['week_name'] == 'Weekday']
        weekendQ3 = final[final['week_name'] == 'Weekend']
        final = footfall_2024Q3.copy()
        typicalQ3 = final.groupby(['year','hex_id']).agg(
            Daytime_sum = ('6am-6pm','mean'),
            Nighttime_sum = ('6pm-6am','mean')
        ).reset_index()
        export_to_csv(typicalQ3)

In [31]:
# quarterly_changes('2024-06-30', '2024-10-01', 3)
quarterly_changes('2024-03-31', '2024-07-01', 3)

Data copied
Successfully exported None to CSV
